<a href="https://colab.research.google.com/github/sivaratrisrinivas/ttt-playground/blob/main/notebooks/all_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/sivaratrisrinivas/ttt-playground/blob/main/notebooks/all_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TTT Playground - All Tests

Combined notebook for all phase tests. Sections:
1. **Setup** - Clone, install, verify GPU
2. **Phase 2** - Document Processing (PDF, Chunker, Validator)
3. **Phase 3** - TTT-Linear Layer

---
# 1. Setup

In [1]:
# Clone repo (or pull latest if exists)
import os
if os.path.exists('/content/ttt-playground'):
    !cd /content/ttt-playground && git pull
    %cd /content/ttt-playground
else:
    !git clone https://github.com/sivaratrisrinivas/ttt-playground.git
    %cd ttt-playground

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd()))
print(f"✓ Working directory: {os.getcwd()}")

Cloning into 'ttt-playground'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 139 (delta 65), reused 87 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (139/139), 66.24 KiB | 1.03 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/ttt-playground
✓ Working directory: /content/ttt-playground


In [2]:
# Install dependencies
!pip install -q -r requirements.txt
print("✓ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.4 MB/s eta 0:00:00
✓ Dependencies installed


In [3]:
# Verify GPU
!nvidia-smi
import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Sun Jan 11 20:28:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Verify all imports
import torch
import transformers
import fitz  # PyMuPDF
import gradio
import tiktoken
import tqdm
from loguru import logger
import pydantic
print("✓ All imports successful!")

✓ All imports successful!


---
# 2. Phase 2: Document Processing

In [5]:
# Import document processing modules
from src.document.pdf_parser import PDFParser, PDFExtractionError
from src.document.chunker import DocumentChunker
from src.document.validator import DocumentValidator
from src.config import DocumentConstraints, DocumentChunk
from transformers import AutoTokenizer
import fitz
print("✓ Document processing imports successful")

✓ Document processing imports successful


## 2.1 Generate Test PDFs

In [6]:
def create_test_pdf(filename: str, num_pages: int, text_per_page: str):
    """Create a test PDF with specified pages and text"""
    doc = fitz.open()
    for i in range(num_pages):
        page = doc.new_page()
        page.insert_text((50, 50), f"Page {i+1}")
        page.insert_text((50, 100), text_per_page)
    doc.save(filename)
    doc.close()
    print(f"Created {filename} ({num_pages} pages)")

text_short = "This is a short test document. " * 50
create_test_pdf("test_short.pdf", 3, text_short)

text_medium = "This is a medium test document with more content. " * 100
create_test_pdf("test_medium.pdf", 20, text_medium)

with open("test_corrupt.pdf", "wb") as f:
    f.write(b"not a valid pdf file")

print("\n✓ Test PDFs created")

Created test_short.pdf (3 pages)
Created test_medium.pdf (20 pages)

✓ Test PDFs created


## 2.2-2.3 PDFParser Tests

In [7]:
parser = PDFParser()

# Test valid PDF
with open("test_short.pdf", "rb") as f:
    pdf_bytes = f.read()

text, page_count = parser.parse(pdf_bytes)
print(f"✓ Parsed test_short.pdf:")
print(f"  - Pages: {page_count}")
print(f"  - Text length: {len(text)} chars")
assert page_count > 0 and len(text) > 0

# Test error handling
try:
    with open("test_corrupt.pdf", "rb") as f:
        parser.parse(f.read())
    assert False, "Should have raised PDFExtractionError"
except PDFExtractionError:
    print("✓ Error handling works")

✓ Parsed test_short.pdf:
  - Pages: 3
  - Text length: 374 chars
✓ Error handling works


## 2.4-2.6 DocumentChunker Tests

In [8]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
chunker = DocumentChunker(tokenizer, chunk_size=2048)
print(f"✓ Chunker initialized with chunk_size={chunker.chunk_size}")

# Test short text (single chunk)
short_text = "This is a short text. " * 10
chunks_short = chunker.chunk(short_text)
print(f"✓ Short text: {len(chunks_short)} chunk(s)")

# Test large text (multiple chunks)
large_text = "word " * 5000
chunks_large = chunker.chunk(large_text)
print(f"✓ Large text (~5000 tokens): {len(chunks_large)} chunks")
for i, chunk in enumerate(chunks_large):
    assert chunk.token_count <= 2048, f"Chunk {i} exceeds limit"

# Verify token preservation
original_ids = tokenizer.encode(large_text, add_special_tokens=False)
reconstructed_ids = []
for chunk in chunks_large:
    reconstructed_ids.extend(chunk.token_ids)
assert reconstructed_ids == original_ids, "Token preservation failed!"
print(f"✓ Token preservation verified: {len(original_ids)} tokens")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (5001 > 2048). Running this sequence through the model will result in indexing errors


✓ Chunker initialized with chunk_size=2048
✓ Short text: 1 chunk(s)
✓ Large text (~5000 tokens): 3 chunks
✓ Token preservation verified: 5001 tokens


## 2.7 DocumentValidator Tests

In [9]:
validator = DocumentValidator()

with open("test_short.pdf", "rb") as f:
    pdf_bytes = f.read()

# Test valid (relaxed constraints)
is_valid, _ = validator.validate(pdf_bytes, DocumentConstraints(min_tokens=50))
assert is_valid, "Should pass relaxed validation"
print("✓ Valid PDF passes")

# Test max_pages violation
is_valid, msg = validator.validate(pdf_bytes, DocumentConstraints(max_pages=2, min_tokens=50))
assert not is_valid
print(f"✓ max_pages violation detected: {msg}")

# Test min_tokens violation
is_valid, msg = validator.validate(pdf_bytes, DocumentConstraints(min_tokens=500))
assert not is_valid
print(f"✓ min_tokens violation detected: {msg}")

# Test corrupt PDF
with open("test_corrupt.pdf", "rb") as f:
    is_valid, msg = validator.validate(f.read(), DocumentConstraints(min_tokens=50))
assert not is_valid
print(f"✓ Corrupt PDF rejected: {msg}")

print("\n" + "="*50)
print("✓ ALL PHASE 2 TESTS PASSED!")
print("="*50)

✓ Valid PDF passes
✓ max_pages violation detected: Page count (3) exceeds maximum (2)
✓ min_tokens violation detected: Estimated token count (93) below minimum (500)
✓ Corrupt PDF rejected: Invalid PDF: Failed to extract text from PDF: Failed to open stream

✓ ALL PHASE 2 TESTS PASSED!


---
# 3. Phase 3: TTT-Linear Layer

## 3.1 Import models package

In [10]:
from src.models import *
print("✓ Step 3.1: from src.models import * succeeds")

✓ Step 3.1: from src.models import * succeeds


## 3.2 TTTLinear.__init__

In [11]:
from src.models.ttt_linear import TTTLinear

layer = TTTLinear(768, 2048, 768)
print(f"✓ TTTLinear instantiated")
print(f"  W_h.shape: {layer.W_h.shape}")
assert layer.W_h.shape == (2048, 768)
print("✓ Step 3.2: W_h.shape == (2048, 768) verified")

✓ TTTLinear instantiated
  W_h.shape: torch.Size([2048, 768])
✓ Step 3.2: W_h.shape == (2048, 768) verified


## 3.3 TTTLinear.forward (inference mode)

In [12]:
import torch

x = torch.randn(1, 128, 768)
y = layer(x, learning=False)
print(f"  Input shape: {x.shape}")
print(f"  Output shape: {y.shape}")
assert y.shape == (1, 128, 768)
print("✓ Step 3.3: Output shape [1, 128, 768] verified")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x768 and 2048x768)

## 3.4 Initial weights stored for reset

In [ ]:
assert hasattr(layer, '_W_h_initial'), "Missing _W_h_initial attribute"
assert torch.allclose(layer.W_h, layer._W_h_initial)
print("✓ Step 3.4: _W_h_initial stored and matches W_h")

## 3.5 TTTLinear.forward (learning mode)

In [ ]:
layer = TTTLinear(768, 2048, 768)
w_before = layer.W_h.clone()

x = torch.randn(1, 128, 768)
y = layer(x, learning=True)

assert not torch.allclose(layer.W_h, w_before), "W_h should change after learning=True"
print("✓ Step 3.5: W_h differs from initial after learning=True")

## 3.6 reset_weights()

In [ ]:
layer.reset_weights()
assert torch.allclose(layer.W_h, layer._W_h_initial)
print("✓ Step 3.6: reset_weights() restores initial W_h")

## 3.7 get_weight_delta()

In [ ]:
layer = TTTLinear(768, 2048, 768)
x = torch.randn(1, 128, 768)
layer(x, learning=True)

delta = layer.get_weight_delta()
print(f"  Weight delta: {delta}")
assert delta > 0
print("✓ Step 3.7: get_weight_delta() > 0 after learning")

## 3.8 Gradient flow

In [ ]:
layer = TTTLinear(768, 2048, 768)
x = torch.randn(1, 128, 768, requires_grad=True)
y = layer(x, learning=False)
loss = y.sum()
loss.backward()

assert x.grad is not None
print("✓ Step 3.8: Gradient flows through layer")

print("\n" + "="*50)
print("✓ ALL PHASE 3 TESTS PASSED!")
print("="*50)